# 리디북스
## 작품 검색 후 데이터 추출

In [2]:
import requests
from bs4 import BeautifulSoup 

##############################################################

def get_info(url, book_id):
    print(url)
    print("리디북스 ID no. " + book_id)
    print("\n-------------------------\n")
    
    req = requests.get(url)
    source = req.text
    soup = BeautifulSoup(source, 'html.parser')

    # 제목
    ridi_title_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div.info_title_wrap > h3")
    ridi_title = ridi_title_list[0].text
    title = ridi_title[5:] #'[연재] '가 제목 앞에 붙음

    # 저자
    ridi_author_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_writer > span > a")
    author = ridi_author_list[0].text

    # 출판사
    ridi_publisher_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.file_info.publisher_info > a")
    publisher = ridi_publisher_list[0].text

    # 총 연재화수 - 정수
    ridi_book_count_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_info_series_complete_wrap > span.metadata_item.book_count")
    ridi_book_count = ridi_book_count_list[0].text
    ridi_book_count = ridi_book_count[2:len(ridi_book_count)-1]
    total = int(ridi_book_count)

    # 완결 여부
    ridi_complete_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p.metadata.metadata_info_series_complete_wrap > span.metadata_item.not_complete")
    if ridi_complete_list[0].text == '미완결':
        complete = '미완결'
    else:
        complete = '완결'

    # 출간일
    ridi_published_date_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.Header_Metadata_Block > ul > li.Header_Metadata_Item.book_info.published_date_info > ul > li")
    ridi_published_date = ridi_published_date_list[0].text
    pub_date = ridi_published_date[2:13]
    
    '''
    # 최근 업데이트일
    ridi_recent_update_list = soup.select("#SeriesListWrap > ul.module_compact_book_list.white_theme.js_compact_book_list > li > div > div > div.table_cell.book_info > div > ul > li.info_reg_date")
    ridi_update = ridi_recent_update_list[ridi_book_count-1].text
    update = ridi_update[6:16]
    '''

    # 표지 사진
    ridi_img = soup.find('div',{'class','thumbnail_image'})
    ridi_img = ridi_img.find('img')
    img_src = ridi_img.get("src")

    # 카테고리
    ridi_category_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > p > a")
    genre = ridi_category_list[1].text

    '''
    # 별점 점수
    ridi_star_rate_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p > span > span.StarRate_Score")
    star_rate = ridi_star_rate_list[0].text

    # 별점 참여자
    ridi_star_rate_count_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_info_wrap > div > p > span > span.StarRate_ParticipantCount")
    star_rate_count = ridi_star_rate_count_list[0].text

    # 관심도
    ridi_preference_list = soup.select("#page_detail > div.detail_wrap > div.detail_body_wrap > section > article.detail_header.trackable > div.header_thumbnail_wrap > div.header_preference > button > span > span.button_text.js_preference_count")
    preference = ridi_preference_list[0].text
    '''

    # 책소개
    ridi_introduction_list = soup.select("#introduce_book > p")
    for j in ridi_introduction_list:
        ridi_introduction = j.text
        if(ridi_introduction.find("<[연재] " + title + ">") >= 0):
            ridi_introduction = ridi_introduction.replace("<[연재] " + title + "> ","")
            ridi_introduction = ridi_introduction.replace("\r","")
            ridi_introduction = ridi_introduction.replace("책 소개","")
            intro = ridi_introduction.replace("펼쳐보기 ","")
            break;

    ############################## 결과 출력 #############################
    
    print("제목: " + title)
    print("리디북스 ID : " + book_id)
    print("저자: " + author)
    print("장르: " + genre)
    print("출판사: " + publisher)
    print("연령 등급: 추후 추가") # 변수명 : age
    print("독점 여부: 추후 추가") # 변수명 : exclusive | 추후 DB 통해 확인 예정
    print("총 연재화수: ", total)
    print("출간일: " + pub_date)
    print("완결 여부: " + complete)
    print("표지 사진 url: " + img_src)
    print("소개: " + intro)

#################################################################################

ridi_base = "https://ridibooks.com/search/?q="
ridi_work = input("작품 검색 : ")
print("\n-------------------------\n")
ridi_search = ridi_base + ridi_work.replace(" ","+")
print(ridi_search)

req = requests.get(ridi_search)
source = req.text
soup = BeautifulSoup(source, 'html.parser')

url_list = soup.select("#__next > main > section > ul > li > div > h3 > a")
check = 0

for i in url_list:
    if(i.text.find("연재") >= 0):
        # 리디북스 작품 ID 추출 : type <string>
        ridi_bookID = i.get("href")
        ridi_bookID = ridi_bookID.split('?')[0]
        ridi_url =  "https://ridibooks.com" + ridi_bookID
        ridi_bookID = ridi_bookID.split('/books/')[1]
        
        # 데이터 추출
        get_info(ridi_url, ridi_bookID)
        check = 1
        break;

# 검색 결과 출력
if check == 0:
    print("검색 결과가 없습니다.")
    
else:  
    print("\n-------------------------\n\n검색 결과 출력 완료")

작품 검색 : 마운드의 마에스트로

-------------------------

https://ridibooks.com/search/?q=마운드의+마에스트로
https://ridibooks.com/books/425162061
리디북스 ID no. 425162061

-------------------------

제목: 마운드의 마에스트로
리디북스 ID : 425162061
저자: 비도오고
장르: 현대 판타지
출판사: 문피아
연령 등급: 추후 추가
독점 여부: 추후 추가
총 연재화수:  137
출간일: 2020.05.15.
완결 여부: 미완결
표지 사진 url: //img.ridicdn.net/cover/425166265/xxlarge
소개: 좌완 파이어볼러, 메이저리그를 넘어 HOF까지, MLB를 제패한다!

-------------------------

검색 결과 출력 완료


# 조아라
## 작품 검색 후 데이터 추출

In [4]:
import requests
from bs4 import BeautifulSoup

def get_info(url, book_id):
    print(url)
    print("조아라 ID no. " + book_id)
    print("\n-------------------------\n")
    req = requests.get(url)
    source = req.text
    soup = BeautifulSoup(source, 'html.parser')

    # 제목
    joa_title_list = soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong > em > a")
    title = joa_title_list[0].text

    # 저자
    joa_author_list = soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong > span > span")
    author = joa_author_list[0].text

    # 총 연재화수 - 정수
    joa_book_count_list = soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.work_view > span.select > span")
    joa_book_count = joa_book_count_list[0].text
    joa_book_count = joa_book_count.replace("편","")
    total = int(joa_book_count)

    # 완결 여부
    joa_complete_list = soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info > strong")
    if joa_complete_list[0].text == '마지막 연재일':
        complete = '미완결'
    else:
        complete = '완결'

    # 출간일
    joa_published_date_list = soup.find("table")
    joa_published_date_list = joa_published_date_list.find_all("tr")
    joa_published_date_list = joa_published_date_list[total]
    joa_published_date_list = joa_published_date_list.find_all("a")
    pub_date = joa_published_date_list[2].text
    
    '''
    # 최근 업데이트일
    joa_recent_update_list = soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info > span")
    update=joa_recent_update_list[0].text
    update=update[:10]+'.'
    '''

    # 표지 사진 url
    joa_img = soup.find("div")
    joa_img = soup.find(class_="img_s")
    joa_img = joa_img.find("img")
    img_src = joa_img.get("src")

    # 장르
    joa_category_list = soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > strong")
    joa_category = joa_category_list[0].text
    joa_category = joa_category.split(' ')[0].replace("[","")
    genre = joa_category.replace("]","")

    '''
    # 조회수 - 정수
    count_list=soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.box_sty01 > div > div > div.txt_c_sty01 > div.info_c > p.info2 > span")
    views=count_list[0].text
    views=views.replace(",","")
    views=int(views)

    # 추천수 - 정수
    recommend=count_list[2].text
    recommend=recommend.replace(",","")
    recommend=int(recommend)

    # 관심도 - 정수
    preference=count_list[4].text
    preference=preference.replace(",","")
    preference=int(preference)
    '''

    # 책소개
    joa_introduction_list = soup.select("#premium_content > div.latest > div.best_list_list_wrap > div.t_cont_v")
    joa_introduction = joa_introduction_list[0].text
    joa_introduction = joa_introduction.replace("\t","")
    intro = joa_introduction.replace("\r","")
    #introduction= introduction.replace("\n","")
    
    # 출판사
    publisher = '미제공'

    ############################## 결과 출력 #############################

    print("제목: " + title)
    print("조아라 ID : " + book_id)
    print("저자: " + author)
    print("장르: " + genre)
    print("출판사: " + publisher)
    print("연령 등급: 추후 추가") # 변수명 : age
    print("독점 여부: 추후 추가") # 변수명 : exclusive | 추후 DB 통해 확인 예정
    print("총 연재화수: ", total)
    print("출간일: " + pub_date)
    print("완결 여부: " + complete)
    print("표지 사진 url: " + img_src)
    print("소개: " + intro)

##########################################################################

joara_base="http://www.joara.com/search/search.html?sl_search=book&sl_keyword="
joara_work=input("작품 검색 : ")
print("\n-------------------------\n")
joara_search=joara_base+joara_work.replace(" ","+")

req=requests.get(joara_search)
source=req.text
soup=BeautifulSoup(source, 'html.parser')

url_list=soup.select("#content > div.view > div.layout > div.series")
check=0
for i in url_list:
    # 프리미엄 카테고리 검색
    if(i.find("h3").text.find("프리미엄")>=0):
        joara_list=i.find("div")
        joara_list=joara_list.find("a")
        joara_list=joara_list.get("href")
        joara_workID=joara_list.split('=')[1]

        # url 링크 추출
        joara_url="http://www.joara.com/premium_new/book_intro.html?book_code="+joara_workID
        get_info(joara_url, joara_workID)
        check=1
        break;

if check==0:
    print("검색 결과가 없습니다.")
else:
    print("\n-------------------------\n\n검색 결과 출력 완료")

작품 검색 : 회귀했더니 입대 전날

-------------------------

http://www.joara.com/premium_new/book_intro.html?book_code=1385782
조아라 ID no. 1385782

-------------------------

제목: 회귀했더니 입대 전날
조아라 ID : 1385782
저자: 작가에바트리체
장르: 판타지
출판사: 미제공
연령 등급: 추후 추가
독점 여부: 추후 추가
총 연재화수:  484
출간일: 19/07/31
완결 여부: 미완결
표지 사진 url: http://cf.joara.com/literature_file/20190731_093909.jpg_thumb.png
소개: 
어렵게 잡은 회귀의 기회.
그런데 입대 전날로 회귀해버렸다.



-------------------------

검색 결과 출력 완료


# 리디북스 작품 리스트 - csv 정리

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 리디북스 작품 리스트
ridi_data = []
total = 0

# [카테고리 : 로맨스 연재]
romance_page = 0
romance_count = 0
while True:
    romance_page += 1
    url = 'https://ridibooks.com/category/books/1650?page=' + str(romance_page)
    print(url)
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 제목 위치
    title_list=soup.select("#page_category > div.book_macro_wrapper.js_book_macro_wrapper > div > div.book_metadata_wrapper > h3 > a > span")

    # 한 페이지 당 작품 최대 20개
    for num in range(20):
        try:
            ridi_data.append([])
            title=title_list[num].text.replace("\n","")
            title = title.split() # 제목 추출
            del title[0] # [연재] 제거
            title = " ".join(title)
            ridi_data[total].append(title) # 리스트에 추출 제목 데이터 삽입
            ridi_data[total].append('리디북스') # 연재 플랫폼 : 리디북스
            print(title)
            romance_count += 1
            total += 1
        except:
            break
    
    print("\n-----------------------------------------------\n")
    
    # 마지막 페이지 확인
    page_list=soup.select("#page_category > div.module_paging > ul > li.btn_next")
    if not page_list:
        break
        
# [카테고리 : 판타지 연재]
fantasy_page = 0
fantasy_count = 0
while True:
    fantasy_page += 1
    url = 'https://ridibooks.com/category/books/1750?page=' + str(fantasy_page)
    print(url)
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 제목 위치
    title_list=soup.select("#page_category > div.book_macro_wrapper.js_book_macro_wrapper > div > div.book_metadata_wrapper > h3 > a > span")

    # 한 페이지 당 작품 최대 20개
    for num in range(20):
        try:
            ridi_data.append([])
            title=title_list[num].text.replace("\n","")
            title = title.split() # 제목 추출
            del title[0] # [연재] 제거
            title = " ".join(title)
            ridi_data[total].append(title) # 리스트에 추출 제목 데이터 삽입
            ridi_data[total].append('리디북스') # 연재 플랫폼 : 리디북스
            print(title)
            fantasy_count += 1
            total += 1
        except:
            break
    
    print("\n-----------------------------------------------\n")
    
    # 마지막 페이지 확인
    page_list=soup.select("#page_category > div.module_paging > ul > li.btn_next")
    if not page_list:
        break
        
# [카테고리 : BL 연재]
BL_page = 0
BL_count = 0
while True:
    BL_page += 1
    url = 'https://ridibooks.com/category/books/4051?page=' + str(BL_page)
    print(url)
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 제목 위치
    title_list=soup.select("#page_category > div.book_macro_wrapper.js_book_macro_wrapper > div > div.book_metadata_wrapper > h3 > a > span")

    # 한 페이지 당 작품 최대 20개
    for num in range(20):
        try:
            ridi_data.append([])
            title=title_list[num].text.replace("\n","")
            title = title.split() # 제목 추출
            del title[0] # [연재] 제거
            title = " ".join(title)
            ridi_data[total].append(title) # 리스트에 추출 제목 데이터 삽입
            ridi_data[total].append('리디북스') # 연재 플랫폼 : 리디북스
            print(title)
            BL_count += 1
            total += 1
        except:
            break
    
    print("\n-----------------------------------------------\n")
    
    # 마지막 페이지 확인
    page_list=soup.select("#page_category > div.module_paging > ul > li.btn_next")
    if not page_list:
        break

        
############################### 결과 출력 ###############################
print("로맨스 장르 총 " + str(romance_count) + "편")
print("로맨스 장르 페이지 : " + str(romance_page))
print("판타지 장르 총 " + str(fantasy_count) + "편")
print("판타지 장르 페이지 : " + str(fantasy_page))
print("BL 장르 총 " + str(BL_count) + "편")
print("BL 장르 페이지 : " + str(BL_page))
print("리디북스 연재작 총 " + str(total) + "편")

############################# csv 파일 생성 #############################
data = pd.DataFrame(ridi_data, columns = ['작품명', '연재 플랫폼'])
data.to_csv('ridibooks.csv', encoding='utf-8', index=False)

print("저장 완료.\n")

https://ridibooks.com/category/books/1650?page=1
포식자의 혼약자
계모인데, 죽음을 피하는 게 너무 쉽다
테라리움 어드벤처
디어 에데르트
낯선여름
가짜 여동생을 원하면
너밖에 모르는
사립학교 이야기
먼치킨들의 총애를 받고 있습니다
파반느
나를 사랑하지 않는 그대에게
악마를 교화하는 방법
라이트 앤 다크
로잔의 가시덤불
상사와 몰래 하는 짓
그림자 없는 밤
내 죽음으로 흑화하지 마세요
내게 복종하세요
악녀에게 의리가 어디 있어
내 악역 남편을 위하여

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=2
흑막 용을 키우게 되었다
배덕 황후
폐하의 밤
미친개가 몽둥이를 물어버리면
더티스캔들
몽중연
파보르 (favor)
엘리트 공주님의 수석지침서
악마가 나를 부르면
약탈혼
가짜 성녀는 퇴장을 기다린다
비비안의 사정
불온한 결속
이제야 나를 그리워한다
폐하가 너무 귀찮게 해
이 결혼은 어차피 망하게 되어 있다
폭군과 여우
인 더 케이지(In the CAGE)
짐승 계약
상수리나무 아래

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=3
아니라도
야수의 성
S급의 히든 퀘스트
홈, 비터 홈
낙원
해피엔딩은 없다
조연은 너나 하세요
악역 대공비의 위험한 이중생활
직장내 불순교제
싫으니까 더
롤 더 본즈(Roll the Bones)
둘이 하는 나쁜 짓
불행의 기원
어느 날 공주가 되어버렸다
도원향
뱀의 혀
스르륵
핸디캡 이론
까마귀는 반짝이는 것을 좋아해
페어 트레이드(fair trade)

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=4
아폴론의 심장
매리지 앤 압생

우린, 사랑일까
완월
오아시스 내 청춘
연애, 하고 있습니까
아랫집 강짱과 윗집 늑대들
살짝궁 손 잡아드립니다
보통연애 환자와 연애할 수 있을까요
보살님은 연애중
바람 이를때면
바나나형수님
밀키러브
몽환 한자락
말썽쟁이 약혼녀
떴다 그녀
달빛연인
늑대, 그녀에게 길들여지다
나의 아름다운 야수
그저, 사랑
그녀에게 올인하다
그 남자, 날씬해진 그 여자의 사정

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=30
NEO, 박씨부인전
붉은 핏빛의 유혹
독과 나비
이계궁녀
목어화
지중해의 검은 표범
타르탄기담
레벨 업 위드 유(Level up with you)
마이 러블리 캡틴
지금 죽으러 가는 길입니다
그날의 약속
사랑할 수 밖에 없는
궁중 스캔들
네가 살아 있으면 좋겠어
구속(외전합본)
프라블럼(Problem)
사랑에 길을 잃다
애정소환
강박증
남자가 되어 돌아온, 그 녀석

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=31
센트럴빌리지의 열대야
그녀의 로맨스는 달달하다
개정판 | 썸씽 (Something)
신경 좀 꺼주실래요?
정심심 Mental Clinic
이중인격 바로잡기
사랑, 불변의 법칙
누님vs형님
꽃같은 증인보호
신데렐라와 바보온달
음침한 베라와 이상한 우주
꽃이 피어날 때
이자벨라 융커
본색
매혹 당한 그녀
황제 폐하 교육은 어떻게 하나요?
캔유 렛미인 ( Can you let me in )
아침이 오지 않아도
웨어울프
야수개

-----------------------------------------------

https://ridibooks.com/category/books/1650?page=32
너를 갖는 건, 음악이 흐른 뒤에
오직 특별한 너
규방로맨스 - 소저의 정인
개정판 | 발코니의 그 

주인공이 자비를 숨김
회귀자의 스타트업
나 혼자 대리사냥!
정감록
권능의 마갑
나 혼자 두배!
거침없이
다크로드 블리츠 (연재)
독표무적 (연재)
각성자
로드는 로드 중
센 놈!
슬기로운 연예생활
재벌을 향해 Suit Up!
철종
먹지 못했던 여사친들
역대급 야구 천재의 탄생
BJ대마도사
무신탄생
마이스터 - Prophet

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=24
나 혼자 천재 DNA
무한 회귀의 용사
회귀로 압도한다
대물 스카우터
링커 : 유물의 주인을 찾아드립니다
천재 광대, 톱스타되다!
불장난
골프가 좋아
500년 만에 돌아온 천마
회귀한 톱스타의 힐링라이프
최강전설 아저씨
하루가 두 번
플레이어님 귀환하신다!
월드스타 슈베르트
학자인데 너무 강함
톱스타, 그 자체
나는 폭군이다
투사의 탈을 쓴 모차르트
탈모형 미드필더
오크지만 찬양해

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=25
능력자의 포스
생명력 Max!
강남 퇴마사
폭풍의 한반도
똥볼 투수의 귀환
부검의 신 : 망자의 기억
만렙 영웅님께서 귀환하신다!
주인공이 모르는 게 없음
SSS급 마트 점원
공작가 막내아들은 다재다능
세계는 넓고 할 일은 많지만 서자였다
축구천재 귀화하다
중원이 지다
이 맛에 재벌한다.
무협지 엑스트라가 되다
재벌가 망나니입니다만?
현대 네크로맨서 길라잡이
21세기 갑부[甲富]
용돈이 계속 늘어!
모든 것을 강화하는 헌터

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=26
게임 같은 무인도 생존기
천마재혼: 회귀로 모든 것을 얻다
배틀 킹덤 언플의 킹
대남궁세가
나 혼자 현실 패치
육군

졸부 집 망나니
내 무공 넘사벽!
잠룡전기(潛龍傳奇)
당신의 매니저
글의 신을 향하여
우주게임의 사령관
돌아온 기간트 마스터
대물남 오천국
사상 최강의 에이스
기연무적
축구를 너무 잘함
뽑기로 강해진 SSS급 헌터
광고천재의 회귀생활
사자의 귀환
개정판 | 무림 드래곤
SSS급 마왕의 귀환
헬무트
나만이 무공을 사용할 수 있다!
튜토리얼 라이프
나 빼고 다 젊은이

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=51
나 혼자 무림 플레이어
마인예속
백작가 도련님은 회귀자
연기를 너무 잘함
야구가 너무 쉬움
이세계 최면 유희
검신, 던전 출수
탑 아이돌 락커를 꿈꾸다
SSS급 촉수괴물이 지능을 얻음!
여초 삼국지
전생 라이프
이과장은 탑스타!
재벌 닷컴
변혁 1990
무적선생
삼국지 위현전 - 죽림결의
전생에 음악의 신이었던 남자
알브레히트 일대기
수양대군, 코끼리를 만나다!
백룡의 비상

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=52
공작가의 소드마스터
페로몬맨
자고나니 세상이 게임으로 바뀌었다!
농구가 보인다!
블러드 씨커
만렙in현실
조선군벌
사마지존(邪魔至尊)
악역 바스타드입니다만
레벨업만이 살길
FFF급 구단 매니지먼트
경험치 포식자
닥터 최태수
나 혼자 통조림 사러 멀리 갔다 왔다
나 혼자 10만 대군
미래의 통장으로 재벌 되다
한냥 표국
마운드의 제왕
차원 이동으로 재벌된 남자
열혈 닥터, 명의를 향해!

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=53
이것이 법이다
3루의 귀신
눈 떠 보니 슈퍼스타
섹신!
난 대물이다
덮쳐놓고 레벨업
귀환자님 뭐 하세요?
그리스의 성신 

파이어볼러
신의 마구
슬러거 프로젝트
패전처리, 회귀하다.
레벨업 에이스
자명공자
이 헌터 실화냐?
[개정판] 레필리아 레소드
작가, 전설을 쓰다
역대급 마법 재능
귀신잡고 폭렙업
악마의 오른발
죽었다 깨어난 가수
비천신마
엘로스의 군주
천하제일가
내 눈에 라인이 보여
내 것만 엑스라지
미소녀를 공략하자
스킬 대장장이

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=79
갓코인
등급인생
화산권마
삼국수호전
리블 온라인
이세계 골드리치
경험 많은 뉴비
쪼렙 천마
개정판 | 페널티박스의 폭군
이세계 영주 생활
야망검
국민배우 정도현
계약이 완료되었습니다
레벨 업! 하는 식당
신흥재벌
믿음직한 1승 카드
이세계 쉐프 생존기
영혼을 노래하다
내가 슈퍼맨이 되던 날
되찾은 새벽

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=80
진짜 쩐다
기록을 부수는 타자
내가 지존이다
회귀군주
천하제일 부자
포션으로 무한성장
재능의 책
무신귀환
신검화산
풍뢰도
괴물 투수 성낙기
7번째 환생
포효강호
회귀했더니 검신이었다
이계에서 무기상
시바! 혼자 딜탱 다 하잖아!
나 혼자 극한 던전
천애협로
마도 진조휘
배우, 미친 흡입력

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=81
탑 레시피가 보여!
금검풍운(金劍風雲)
스크린의 별
천 번의 환생 끝에
전생부터 다시
오버로드 : 남들은 헬 나에겐 이지
일인전승(一人全勝)
고검독보
드래곤 레이드
신 조선:개혁의 파도
환웅의 검
다시 한 번
물약이 자꾸 땡겨
만석꾼
내 몸은 무한재생
회귀자의 스킬무쌍
마검쟁천
두 세계의 재벌
최강질주
이세계 플레이어

---------

심장 강탈자
이계의 후예
건축의 신
링커
강남점쟁이
현세독존기
혈황지담
퓨어 헌터
패자지천
패왕전설
판광천하
탑으로 세계정복
킬러의 이계는
치트키 쓰는 재벌님
정령사
적룡마제
이계용사 귀환기
의생진검
내 인생 최고의 샷
심연

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=108
승승장구 9서클 마스터
스타셰프, 분식점 아들되다
술로 세상을 얻다
소환왕 강철중
소공자로 환생한 그랜드 마스터
삼류작가 회귀하다
보스 회귀하다
몰락무당
더 하이브
대군주 하선
당문소공자 당사독
나쁜 재벌
기간트 마스터
군림무적
공동전기
곤륜소협
곤륜금선
고수 현대생활백서
개미왕
북유럽 신화

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=109
꼼짝 안 해도 레벨업
환생표국
포인트 헌터
뽑기의 신이 강림하였습니다.
나는 뱀파이어다
검성, 환생하다!
전생괴의
몬스터는 로또다
더 문 차원의 비밀
헌터, 웃기지마!
무적호위
신체 주고 게임 최강
주먹쓰는 사제님
황혼. 직감의 소유자
브리튼의 용병 군주
귀신 보는 퇴마사
삼국지 - 패왕지략
창공의 기사
일대검호
남천문

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=110
21세기 신마록
명왕전기
천인혈
파멸왕
환영무인
대붕신화
내 프리킥이 너무 잘 들어간다
검마도
인챈트로 인생역전!
팽가신화
스타 작가 차태훈
제국의 꿈
에이스 카드(ACE CARD)
주먹의 품격
업어 키운 BJ
판사 이한영
스낵의 왕
우리 삼촌은 월드스타
권왕전생
불멸의 슬레이어

-----------------------------------------------

https://ridibooks.co

낙오무사
야수학개론
총수 : 수를 읽는 남자
마계를 삼키다
무림으로 간 바리스타
삼국지 강유전
레벨만 높으면 다냐
초인기
강철대제
디데이즈
프로 농사꾼의 귀환
치트키의 제왕
현대 귀환 강화사
귀검무영
꿈만 꾸면 강해져
마석 포식자
스펙의 신
IT제국
귀환 던전 클리너
격투의 신

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=140
좀비버스터
영웅학개론
후회는 없다
부마 신익성
프로야구 마스터
금수저 입에 물고
야구 마스터
펄스나인
정도제일마존문
지장대공
후작가 둔재는 먼치킨
다시 한 번 톱스타
시간 상점으로 나도 역대급 재벌
대한민국 천재
유대인과 세계경영 재벌
외계 기생체의 숙주
재능복사 미드라이너는 프로게이머
마운드의 마에스트로
천재캐디를 위하여
이계에서 돌아온 대통령

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=141
아포칼립스 2회차
승승장구 연예계 생활
회귀적 반로환동
첩자인데 천마
천마현신(天魔現身)
무적합체 김봉팔
등룡비호
21세기 절대자
시작부터 무쌍
먹이사슬 최상위의 포식자
조선에서 귀환했다!
만능 스마트폰으로 인생역전!
돈버는 속도 초당3억
나의 검으로 너를 살다
신의 조각을 품은 남자
전장의 악귀들 : 우리도 내정한다
패황성(覇皇城)
전사(戰士) 강철산(姜鐵山)
미래를 훔쳐라
신화를 쓰는 대공

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=142
맹주의 귀환
흑천의 칼이 울어
게이밍 헌터
수호기사(Guard Knight)
나는 군단이다!
게이트 재벌
향기: 거부할수 없는 유혹
천마! 이계를 부수다
영웅 따위 너나 해라
삼구삼진
북벌군주 효종 환생
꿈의 전설
금서고의

악역무쌍
정령 포식자
주방의 마법사(개정판)
노템! 최강 헌터
개정판 | 전설이 돼라
개정판 | 진짜 사나이
티끌 모아 대박
이계로 돌아갈래
타임 리미트
영주의 전성시대
황제의 레벨 업
현자귀환
기획에 산다
술사귀환
군주, 다시 귀환하다
헌터 킹
더 캡틴
바보가 만드는 멋진 세상
카르마 시스템
엠페러 오브 매직

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=168
코트 위의 지배자
디멘션 - 게임의 왕
매지컬 써전
불사단-체력의 제왕
천마전생
나는 명왕이다
매니지먼트의 신
이세계의 정령사가 되었다
날로 먹는 플레이어
미래를 고르는 매니저
회귀해보니 흑마법사
엑스트라로 꿀 빠는 법
너무 강한 산장 주인
독식 랭커의 회귀
나는 무신이다
의원귀환
현실에서 내가 해적이라니
해부학과 학생은 게임중
나 혼자 무공창조 in게임
나 홀로 주문 사용자

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=169
나는 흑마법 작가다
뭐든지 만드는 랭커 플레이어
치트쓰고 시작하는 영주생활
망캐로 클리어하는 방법
무공 익힌 영주님
9클래스 흑마도사, 회귀하다
하이마스터 (개정판)
이세계에서 정령들과 노래하고 있습니다!
나는 무공 배운 드래곤이다
처음 사제로 전직했습니다
회귀 마도사의 일상
최고의 맛을 요리해드립니다
SSS급 뽑기 플레이어
천마님 요리하신다
검의 군주는 평온을 원한다
마법사 이계 여행기
진화학개론
용사의 C급 동료
살인기계
버프받고 탑스타

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=170
회귀로 지존이 된다
좌도방문
랭킹 1위는 마을사람
워크 마스터
정령군주
테페른의 영주
절대 검신이 된다
전직 

무한각성
무명창
무림신녀
개정판 | 무림숙수 셰프되다
무당학사
무당제일 문지기
무당권왕
메이드 인 드워프
만부부당
마탑의 사냥개
마도시대 마장기
마도사, 고려에 떨어지다
마도교육 마스터
마교 소교주
마교 소공자
리플레이스
리얼리더
로또 인생역전
로드 오브 나이트
돈 주고 이계를 사 버렸다

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=198
데우스 프로그램
데빌스 웨이
던전의 주인
더블라이프 : 액터
더 스파이더
대륙지존기
대도 루피옹
대격변
내가 믿는 것은 검
나는 마초다
나 혼자 이세계 플레이어
기적에 걸어라
기사의 신격
그랜드 체어맨
그랜드 마스터의 귀환
그 현자는 히키코모리의 삶을 꿈꾼다
귀환마스터
귀환
굴러들어온 최강의 사역수
구름공작

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=199
곤륜장문
계약직 용사
검제
검은 꽃에 지고
각성 능력자
가문의 마법사
가르딘 전기
미러 나이트
헌터의 던전 라이프
탑스타가 될 운명입니다
신의 게임
무림세가 막내아들로 빙의했습니다
정상(頂上)
초능력은 마법 판정이라 카더라
백룡공작 팬드래건
에고소드 들고 회귀하다
회귀해도 다시 매니저!
The Answer(디 앤서)
그리스 로마 신화
달의 미궁

-----------------------------------------------

https://ridibooks.com/category/books/1750?page=200
호루스의 반지
삼국지 잉여전
마나의 군주
5툴 플레이어(five-tool player)
이미테이션 라이프
개정판 | 로또가 전부는 아니야
블러드
형이 가라사대
연예계의 제왕
나는 군단이다
완전판 | 잠룡대제
신좌의 게임
마왕 전투력 999,999,999,999
삼국지 무극
나는 죽지 않는다
신수마스터

모란기획
시스의 빛
만남과 이별의 변주곡
장르 탈출이 시급하다
너를 부르는 숫자
헌트&헌트(Hunt&Haunt)
이기적인 크리스마스를 보내는 방법
해파리 소년
악마와의 화양연화
투 톱(Two Top)
누군가의 오팔
머피의 법칙
모티마(Motima: Star in the universe)
겨울 나무 숲
세컨드 시크릿 3부 : 왕자님의 비밀
세컨드 시크릿 2부 : 제멋대로 왕자님
두근두근
친구로는 안 되겠니?
친구 새끼, 못난 새끼
제주도의 푸른밤

-----------------------------------------------

https://ridibooks.com/category/books/4051?page=25
씬 스틸러
봄이 왔다
립스틱 바르는 남자
하늘에 울려퍼지는 용의 노랫소리
나쁜남자 앤솔로지
열두 번의 시그니처
순결한 새색시
아스테_Asteroid
마피아와 마법사
신의 연애사

-----------------------------------------------

로맨스 장르 총 649편
로맨스 장르 페이지 : 33
판타지 장르 총 4000편
판타지 장르 페이지 : 200
BL 장르 총 490편
BL 장르 페이지 : 25
리디북스 연재작 총 5139편
저장 완료.



In [8]:
# 파일 확인하기
df_ridi = pd.read_csv('ridibooks.csv')
df_ridi

,작품명,연재 플랫폼
0,포식자의 혼약자,리디북스
1,"계모인데, 죽음을 피하는 게 너무 쉽다",리디북스
2,테라리움 어드벤처,리디북스
3,디어 에데르트,리디북스
4,낯선여름,리디북스
...,...,...
5136,아스테_Asteroid,리디북스
5137,마피아와 마법사,리디북스
5138,신의 연애사,리디북스
5139,NaN,NaN


# 조아라 작품 리스트 - csv 정리

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 조아라 작품 리스트
joara_data = []
total = 0

# [카테고리 : 로맨스 - 현재 연재 중]
joara_rom_page = 0
joara_rom = 0
while True:
    joara_rom_page += 1
    url = "http://www.joara.com/premium_new/book_list.html?page_no=" + str(joara_rom_page) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=&sub_category=&sl_cate_no=fm"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_rom_page -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        title_list = []
        romance = soup.find_all('div',{'class','list_list_con best_c'})
        romance = soup.find_all('div',{'class','finish_list_view'})
        romance = soup.find_all('b')
        
        for i in romance:
            title_list.append(i.get_text())
            
        # 중복 제거
        title_list = list(set(title_list))
        
        if '연재 작품' in title_list:
            title_list.remove('연재 작품')
        if '조회:' in title_list:
            title_list.remove('조회:')
        if '추천:' in title_list:
            title_list.remove('추천:')
        if '선작:' in title_list:
            title_list.remove('선작:')
        if 'Fax' in title_list:
            title_list.remove('Fax')
        if 'Tel' in title_list:
            title_list.remove('Tel')
        
        joara_rom += (len(title_list))
        print(title_list)
        
        # 전체 리스트에 넣기
        if len(title_list) != 0:
            for num in range(0,len(title_list)):
                joara_data.append([])
                joara_data[total].append(title_list[num]) # 리스트에 추출 제목 데이터 삽입
                joara_data[total].append('조아라') # 연재 플랫폼 : 조아라
                total += 1
        
print("\n---------------------------------------------------------------\n")

# [카테고리 : 로맨스 - 완결]
joara_rom_end = 0
while True:
    joara_rom_end += 1
    
    url = "http://www.joara.com/premium_new/finish_list.html?page_no=" + str(joara_rom_end) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=finish&sub_category=&sl_cate_no=r"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_rom_end -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        title_list = []
        romance = soup.find_all('div',{'class','list_list_con best_c'})
        romance = soup.find_all('div',{'class','finish_list_view'})
        romance = soup.find_all('b')
        
        for i in romance:
            title_list.append(i.get_text())
            
        # 중복 제거
        title_list = list(set(title_list))
        
        if '완결 작품' in title_list:
            title_list.remove('완결 작품')
        if '조회:' in title_list:
            title_list.remove('조회:')
        if '추천:' in title_list:
            title_list.remove('추천:')
        if '선작:' in title_list:
            title_list.remove('선작:')
        if 'Fax' in title_list:
            title_list.remove('Fax')
        if 'Tel' in title_list:
            title_list.remove('Tel')
        
        joara_rom += (len(title_list))
        print(title_list)
        
        # 전체 리스트에 넣기
        if len(title_list) != 0:
            for num in range(0,len(title_list)):
                joara_data.append([])
                joara_data[total].append(title_list[num]) # 리스트에 추출 제목 데이터 삽입
                joara_data[total].append('조아라') # 연재 플랫폼 : 조아라
                total += 1
        
print("\n---------------------------------------------------------------\n")

# [카테고리 : 판타지/무협 - 현재 연재 중]
joara_fm_page = 0
joara_fm = 0
while True:
    joara_fm_page += 1
    url = "http://www.joara.com/premium_new/book_list.html?page_no=" + str(joara_fm_page) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=&sub_category=&sl_cate_no=fm"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_fm_page -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        fm_title_list = []
        fantasy = soup.find_all('div',{'class','list_list_con best_c'})
        fantasy = soup.find_all('div',{'class','finish_list_view'})
        fantasy = soup.find_all('b')
        
        for j in fantasy:
            fm_title_list.append(j.get_text())
            
        # 중복 제거
        fm_title_list = list(set(fm_title_list))
        
        if '연재 작품' in fm_title_list:
            fm_title_list.remove('연재 작품')
        if '조회:' in fm_title_list:
            fm_title_list.remove('조회:')
        if '추천:' in fm_title_list:
            fm_title_list.remove('추천:')
        if '선작:' in fm_title_list:
            fm_title_list.remove('선작:')
        if 'Fax' in fm_title_list:
            fm_title_list.remove('Fax')
        if 'Tel' in fm_title_list:
            fm_title_list.remove('Tel')
        
        joara_fm += (len(fm_title_list))
        print(fm_title_list)
        
        # 전체 리스트에 넣기
        if len(fm_title_list) != 0:
            for num in range(0,len(fm_title_list)):
                joara_data.append([])
                joara_data[total].append(fm_title_list[num]) # 리스트에 추출 제목 데이터 삽입
                joara_data[total].append('조아라') # 연재 플랫폼 : 조아라
                total += 1

print("\n---------------------------------------------------------------\n")

# [카테고리 : 판타지/무협 - 완결]
joara_fm_end = 0
while True:
    joara_fm_end += 1
    url = "http://www.joara.com/premium_new/finish_list.html?page_no=" + str(joara_fm_end) + "&sl_category=&sl_search=&sl_keyword=&sl_orderby=&sl_othercategory=&list_type=finish&sub_category=&sl_cate_no=fm"
    
    req=requests.get(url)
    source=req.text
    soup=BeautifulSoup(source, 'html.parser')
    
    # 마지막 페이지 확인
    if (soup.find('div',{'class','list_list_con best_c'})) == None:
        joara_fm_end -= 1
        print("마지막 페이지.")
        break
        
    else:
        # 제목 추출
        fm_title_list = []
        fantasy = soup.find_all('div',{'class','list_list_con best_c'})
        fantasy = soup.find_all('div',{'class','finish_list_view'})
        fantasy = soup.find_all('b')
        
        for j in fantasy:
            fm_title_list.append(j.get_text())
            
        # 중복 제거
        fm_title_list = list(set(fm_title_list))
        
        if '완결 작품' in fm_title_list:
            fm_title_list.remove('완결 작품')
        if '조회:' in fm_title_list:
            fm_title_list.remove('조회:')
        if '추천:' in fm_title_list:
            fm_title_list.remove('추천:')
        if '선작:' in fm_title_list:
            fm_title_list.remove('선작:')
        if 'Fax' in fm_title_list:
            fm_title_list.remove('Fax')
        if 'Tel' in fm_title_list:
            fm_title_list.remove('Tel')
        
        joara_fm += (len(fm_title_list))
        print(fm_title_list)
        
        # 전체 리스트에 넣기
        if len(fm_title_list) != 0:
            for num in range(0,len(fm_title_list)):
                joara_data.append([])
                joara_data[total].append(fm_title_list[num]) # 리스트에 추출 제목 데이터 삽입
                joara_data[total].append('조아라') # 연재 플랫폼 : 조아라
                total += 1

############################### 결과 출력 ###############################
print("로맨스 장르 총 " + str(joara_rom) + "편")
print("로맨스 장르 페이지 : " + str(joara_rom_page))
print("로맨스 완결 페이지 : " + str(joara_rom_end))
print()
print("판타지 장르 총 " + str(joara_fm) + "편")
print("판타지 장르 페이지 : " + str(joara_fm_page))
print("판타지 완결 페이지 : " + str(joara_fm_end))
print()
print("조아라 연재작 총 " + str(total) + "편")

############################# csv 파일 생성 #############################
joara_dt = pd.DataFrame(joara_data, columns = ['작품명', '연재 플랫폼'])
joara_dt.to_csv('joara.csv', encoding='utf-8', index=False)

print("저장 완료.\n")

['치트코치 정수찬', '조지 부시의 위대한 미국', '환생자의 재벌테크', '검사 김서진', '대표님의 향기로운 덕질생활', '3회차 톱매니저', '로그인 더 헌터', '무제 관우전: 천하를 품은 구름', '회귀했더니 입대 전날', '제가 게임 속 악신이라고 하네요', '인생역전 우강호', '나 혼자 검술상점', '북경의 신룡', '축복의 블루스톤', '무당패왕(武當覇王)', '방구석 프로듀서', '주인공이 불행을 다 막음', '나 혼자 미래독식 서바이벌 ', '회귀자의 로열라이프', '나만의 소원퀘스트 ']
['어서 오세요 드라큘의 흥신소에', '블랙기업조선', '신비의 제왕', '163417413번째 소울라이크 용사', '일념영원', '내 능력은 매크로', '피아니스트의 마을', '시스템 강호지존', '이것이 법이다', '극단적으로 축구하는 놈', '엔딩메이커', '카오스맨', '성진변', '멸망한 세계의 농부', '작전명 : 귀향', '몽골에서 유럽까지', '학사신공', '만렙 영웅님께서 귀환하신다!', '재각성을 하고 보니 마왕이 되었다', '타자가 공을 너무 잘 던짐']
['1999년 게임 스타트', '돌아온 용병왕', '천재의 음악방송', '국적없는 용병', '성스러운 아이돌', '뽑기로 무림최강', '템복사로 인생 역전!', '이번엔 진짜 재벌!', '검신환생', '함장에서 제독까지', '조선철강왕', '알고 보니 천재 뮤지션', '평범한 회사원입니다', '나 혼자 균열에 산다', '막장드라마의 제왕', '신의 무공을 배운 초월급 헌터', '역대 최강의 영주님', '말단 마법공학자의 귀환', '내 딸이 검술 천재', '전략게임 속 엑스트라']
['시스템 에러로 종족초월', '헌터의 차원 농장', '믿고 보는 봉감독님', '시체사냥꾼 [연재]', '역대급 재벌 사위', '회귀자 사용설명서', '로그인하자마자 VIP', '혈마검신', '소설 속 엑스트라', '전직 용사', '70억분의 1의 이레귤러', '0살부터 슈퍼스타', '일신상의 이유로 잠시 

['상태창 소드마스터', '종천지애', '피가 부르는 바람', '힐러, 현대를 주름잡다', '악마를 부리는 소녀', '강호기행록 [개정증보판]', '심장 강탈자', '사이코패스와 기업살인', '사천당문', '신화경', '마도종사', '더 데이 : 태초의 전쟁', '이력서의 경력란에 마왕이라고 적을 수 있..', '민간군사기업  BLACK ANGEL', '오러워즈', '피도 눈물도 없는 용사', '불량용사', '포인트는 넘쳐흐른다고!', '태엽장인', '아프니까 절대자다']
['선혈의 군주', '용제, 신을 찢어라', '편의점의 소드마스터', '싱크[Sync]', '천년 백작: 브라운가의 비밀', '미스터 매직ENT', '게임 캐릭터로 이계에 떨어졌다', '어플-미래 미리보기', '고검독보', '지천명아비무쌍', '삼국지(三國志):더 비기닝', '진실게임 : 배신의 덫', '포춘 콜렉터', '봉황의 비상 - 조일전쟁편', '시작하면서부터 끝나는 그와 그녀들의 학교..', '역대급 먼치킨 마법사의 회귀', '지옥에서 온 좌완 파이어볼러', '검왕까지 레벨업!', '야장, 기적을 만들다', '삼인조(三人組)']
['히든 직업으로 대박내기', '히어로는 없다', '그레이트 써전', '검황이계정벌하다', '대마법사 락스타 (Magical Rockstar)', '히든 메이지', '마장 골핑머신', '하얀 그림자―영웅연대기', '검황도제', '제작해서 갑질한다', '프리랜서 마법사', '임베디드', '대제국시대', '영웅의 발자취 - 두 개의 검', '랍타: 이계 변호사', '말세 종결자', '더블 에이스', '백수연대기', '뽑기의 신이 강림하였습니다', '깃발 아래서']
['전능의 지배자', '제국의 새벽', '영혼을 지배하는 플레이어', '지존', '유령의 주인-유령도서관-', '월드 엔터테인먼트', '재벌로이드 (Chaeboloid)', '바람의 아들', '그라운드의 지배자', '맥스씨', '샤이닝 스타', '꿈을 찾아 왔단다', '얼라이브', '노룡전기', '잠룡

['잔혹하게 농락하다', '안녕, 내 똥강아지', '러브 미 데드', '나의 황금 길들이기(외전 포함)', '첫사랑과 열애 중', '주인공을 죽였습니다', '프러포즈', '시크릿 집사', '그대는, 나의 봄', '구미호에 남자가 없을리가', '고양이는 아홉 번을 산다', '경성탐색 ', '나의 작가님', '간절기', '왜 네게서 벗어날 수 없는가', '오늘 뭐 먹지?', '야누스의 달', '[BL]거인의 오두막', '나의 이름은', '아무것도 모르면서']
['나의 하늘신', '봄을 만들고 있습니다', '귀묘한 카페', '나의 남편으로 명하노라', '그대의 계절', '카모마일 한 모금', '여름 매미 우는 소리', '나는 마왕이로소이다', '규방로맨스', '공녀님 우리 공녀님', '아임 소리, 레드', '[개정판] 미인도(美人圖) : 신수의 여인', 'Mission! 구남친을 피하라', '꿈 그리고 여자', '난폭한 황제 길들이기 [완결]', '사막의 나란토야', '당신은 나의 수면제', '아이슬리드', '또 한 명의 프린세스', '격변의 시대']
['그럼에도 우리는', '신과 결혼했다', '죽여주는 남자', 'Lost Memory', '리온, 이리 오너라!', '너에게 흐르다', '불편한 사랑', '만화 같은 이야기(프렌즈 시리즈 1)', '좋아하는데 이유가 필요해?', '프라이빗 하우스(Private House) 3부', '그를 유혹해(프렌즈 시리즈 2)', '프라이빗 하우스(Private House) 2부', '프라이빗 하우스(Private House) 1부', '신시기묘사', '핫 폴리스', '[동양] 황제의 작은 새 (Clean Ver) 完', '사랑하는 사람들', '사돈지간', '닥치고 결혼', '거짓은 거짓으로']
['[BL]베리테스 연애담', '나이는 숫자에 불과하다', '그리움은 달빛 [완결]', '늑대와 여우 사이', '네가, 웃기를', '밤에 오는 남자', '미남과 야수 [완결]', '벨몬가의 유언장', '홍대 앞 로맨스', '사랑, 그건 아

['톱스타, 별을 만나다', '갈증(Thirst)', '욕정과 욕망의 사이', '공작의 청혼', '만 지 작', '쿠키 드실래요?', '결혼당하다', '라인강의 종소리', '불꽃같진 않아도 괜찮아', '샤를과 베일 속의 주인', '스타의 연인', '손끝에서 마법이', '내 남편의 여자', '인디언 썸머, 우리들의 행복한 계절', '섹시한 요조숙녀', 'Secret Lover', '단테의 낙원(Dante＇s Paradise)', '아파, 당신이라서……', '어느 멋진 날', '설란']
['얼음성의 주인', '사랑 정비 중', '절벽 아래, 매鷹', '르웬의 세계(외전 포함)', '나의 검은 공주님', '여기사와 용', '잔드바르의 여름', '미묘한 사이', '옷장을 열면 그곳에 바다가 ', '영원의 연인', '황태자님 분부대로 (외전증보판)', '눈보다 아름다운*', '악마와 유리구두 (개정판)', '남자의 미학', '사랑니', '인체개조공장', '해가 뜨는 시간', '리스타트(Restart)', '신데렐라 로맨스', '책가방을 멘 신녀']
['적도의 밤', '파트너', '황후의 남자', '카르페디엠', '반짝반짝 (외전증보판)', '미치도록 안고싶은 너', '내 얼굴을 가진 당신', '애이불비', '태권브이와 시바견', '마녀의 귀환 ', '워너비 콤플렉스', '블랙리스트', '달콤한 정복', '사랑한다, 살아간다', '살아남아라, 황후! ', '비밀소녀의 연애사', '그대의 라임라이트', '나는 한그루입니다 ', '블랙 ＆ 화이트', '나는 너를 거부한다']
['귀신같은 그녀', '아내가 돌아왔다', '요아전', '엄마가 되어 주세요', '강철의 꽃', '전 남편 유혹하기', '봄날은 간절히', '찬스', '가면의 미소', '보나페티', '내일은 연애왕', '꽃에 미치다', '아내는 베이비시터', '봄이라 쓰고 곰이라 읽는다', '바라연', '사랑과 미련의 경계선', '깡패오빠와 연애하기', '라스트 레볼루션', '끈적끈적 다가오는', '적대적 우애관계']


['어서 오세요 드라큘의 흥신소에', '블랙기업조선', '신비의 제왕', '163417413번째 소울라이크 용사', '일념영원', '내 능력은 매크로', '피아니스트의 마을', '시스템 강호지존', '이것이 법이다', '극단적으로 축구하는 놈', '엔딩메이커', '카오스맨', '성진변', '멸망한 세계의 농부', '작전명 : 귀향', '몽골에서 유럽까지', '학사신공', '만렙 영웅님께서 귀환하신다!', '재각성을 하고 보니 마왕이 되었다', '타자가 공을 너무 잘 던짐']
['1999년 게임 스타트', '돌아온 용병왕', '천재의 음악방송', '국적없는 용병', '성스러운 아이돌', '뽑기로 무림최강', '템복사로 인생 역전!', '이번엔 진짜 재벌!', '검신환생', '함장에서 제독까지', '조선철강왕', '알고 보니 천재 뮤지션', '평범한 회사원입니다', '나 혼자 균열에 산다', '막장드라마의 제왕', '신의 무공을 배운 초월급 헌터', '역대 최강의 영주님', '말단 마법공학자의 귀환', '내 딸이 검술 천재', '전략게임 속 엑스트라']
['시스템 에러로 종족초월', '헌터의 차원 농장', '믿고 보는 봉감독님', '시체사냥꾼 [연재]', '역대급 재벌 사위', '회귀자 사용설명서', '로그인하자마자 VIP', '혈마검신', '소설 속 엑스트라', '전직 용사', '70억분의 1의 이레귤러', '0살부터 슈퍼스타', '일신상의 이유로 잠시 휴재합니다', '1993 회귀재벌', '뮤리', '개망나니 철종', '경애하는 수령동지', '삼국지 여포전', '1919년 대한광복만세 [연재]', '각성으로 차원최강']
['예능의 제왕', '각성자', '공작님, 회개해주세요!', '내가 키운 S급들', '회귀한 톱스타의 힐링라이프', '저주받아 스펙업', '재능이 자꾸 늘어', '고려 상왕', '알고보니 일대일 천재', '회귀자의 스타트업', '백작가 서자의 가정교사', '회귀자의 그랜드슬램', '농사로 이계 평정', '백작가 장자는 재벌', '조자룡 하북제패

...

['다크킹덤', '천마를 탑재하다', '화염마제', '지배자의 반지', '라스트 킹덤', '더블 게이트', '생사도', '차원의 성주', '야수의 땅', '클레이즈 A.C', '비정소옥', '탑 스타(Top Star)', '게이트 재벌', '집행자', '천군의 꿈', '명랑소년 성공기', '디스트로이', '몽골 사용 설명서', '천위', '스펙테이터']
['메카니스트', '마스터 오브 웨폰', '가면의 마존', '지구를 재활용하다', '다크 프리스트', '가상무공탄류', '오직 나만이 마스터다', '데버스테이션 강진후', '퍼레이션트', '윈드 엠페러', '플라잉버스터', '씨어', '디재스터', '작은 아버지는 동탁 [삼국지 동황전]', '프리매지션', '히든클래스', '미연시인데 연애를 할 수 없는 건에 대하여', '패도무혼', '미래 나노봇을 먹었다', '절대고수']
['루머 읽는 배우님', '상점창 스트라이커', '제온의 후예', '대마법사 회장님', '청소부의 행운 스텟이 폭주한다', '귀환하면 세계 최강', '전신환생', '권능의 마갑', '무종도담', '주먹은 강기를 싣고', '잠자는 용', '나 혼자 천재 DNA', '죽은 자들의 징벌', '절대검황', '암묵의 지배자들(원제 : 불의 고리) [완결]', '재력으로 후려치는 환생 경찰', '골프가 좋아[완결]', '붉은 제국의 축구 황제', '제국 창건기', '이도구검(異道求劍):잘써는놈']
['회귀자의 기연 싹쓸이', '삼구삼진', '미스터 마스터 고산봉', '북벌군주 효종 환생', '시작부터 EX등급', '복수를 위해 돌아왔다', '회귀한 헌터가 너무 강함', '태을검제', '검선지가', '절대자의 푸드트럭', '역전무사', '전기의 기사', '레벨업 건달 강대풍', '전설이 돌아오다', '강민혁, 세계를 먹다', '강철 마법사', '해왕전기', '헨드릭의 시스템', '카드빨로 축구 스타', '부검 스페셜리스트']
['월드 오브 라그나로크', '영주, 황제가 되다', '은영', '오펜스 챌린지

['강자생존', 'SSS급 미친놈이 캐리한다', '카오스 월드', 'vvip 스카우트', '이 남자가 사는 법', '최강 강 대리', 'SSS급 랭커, 회귀하다', '포식하며 폭풍성장 [완결]', '미래를 본다', '검존, 헌터가 되다', '삼국지 - 패왕지략', '영주의 전성시대', '축구가 가장 쉬웠어', '회귀했더니 너무 강해졌다', '바운티 헌터', '특성이 행운이다.', '처음 사제로 전직했습니다', '역린', '배우, 다시 서다', '이 세계는 내가 지배한다']
['상태창이 사라졌다.', '공작가의 마마보이', '내가 뽑은 S급들이 배신했다', '3루의 귀신', '축구는 상태창으로 하는 거야', '승부의 신', '백호의 나라', '길 가다 주운 SSS급 반지', '내 디딤발이 360°', '은둔형 마법사', '무패의 랭커', '제왕의 길', '쾌감 폭발 레벨업', '마계 이야기', '회귀한 소환술사', '100년 수련한 축구선수', '사상 최강의 마법사', '내 상태창 2개', '배우, 조선 왕이 되다', '라스트 콘스탄틴']
['사신전설', '국민아이돌 - 프로듀스99', '귀환전설', '마법도 이력서에 써도 되나요?', '헌터, 웃기지마! [완결]', '나는 활쟁이다', '난검무림', '독표무적', '주식 재벌 박태준', '더 세틀러', '1560 대한민국', '어쩌다보니 이세계에 여우로 전생했습니다.', '블러드 씨커', '아이템 매니아', '최강의 포수', '무림 신세계', '곤왕기', '업어 키운 사매들', '강남점쟁이', '고고학자인데 땅을 안 팜']
['인생럭키! 신유의 만화경', '나 혼자 역대급 수련', '리그 오브 발할라', '탁구왕 김제빵', '달의 미궁', '닥터 최태수', '자명공자', '포탑으로 자동사냥', '진화학개론', 'S급 병사 반세주', '역대급 먼치킨 소환사의 귀환', '맛깔나게 만들어드려요!', '퍼플 미스트', '무림전생(武林轉生)', '만렙인턴 이현성', '백작가 도련님은 회귀자', '풋볼 매니저 : 스탯이 

['에이징(Aging) : 에라이 이 나이에 징집이..', '별神 초신성', '제국의 군인', '게으른 탑스타', '대해적 시대', '셰도우 로드', '백야쟁천', '포텐 터진 배우님', '더 시커 : 마나가 필요 없어', '최고부자', '환생부터 재벌까지', '내 미친 잠재력', '아 잠깐만! 나 또 강해짐', '북천적가', '장자지몽in( 책속의 나와 하나가 되다)', '묵객', '리터니(Returnee)', '엘리트 대작전', '마왕의 표국', '[개정판] Golden Blackhole']
['신들린 뮤지션', '삼국지 마휴전', '스폰', '가문의 주인', '창천', '현대갑질', '마객 신마전설', '만학검전', '기적의 환생', '시스템 스트라이커', '대한제국사 - 한러 전쟁', '전공 삼국지', '용문쟁천', '인피니티메이지', '주먹의 품격', '천마 하고 싶은 거 다 해!', '귀환무사', '맞을수록 강해져', '픽 미 업!', '군주, 다시 귀환하다']
['포텐 폭발, 김작가!', '생존가로 각성하다!', '에이스 카드(ACE CARD)', '내가 조선의 주인이다', '괴물 마법사', '업어 키운 걸그룹', '멸망한 세계의 유일한 마법사', '파워피처 만득씨', 'D등급 개미, SSS등급 되다!', '할리우드 탑스타 ', '앨버트로스', '소환수 레벨 업!', '개방 방주 창업신공', '포제션', '멸망한 세계의 사냥꾼', '데릴사위, 재벌 회장 되다.', '그랜져', '철혈의 회귀자', '미칠듯한 레전드의 존엄', '그림자가 미래를 알려준다']
['아빠는 탑스타!', '낭만용병', '만검조종', '태극검해', '재벌 매니지먼트', '무당괴협전', '삼국지(三國志) [마행처우역거馬行處牛亦去]', '소림곤왕', '수색 조선', '상남자스타일', '드라마의 제왕', '천상의 목소리', '교사는 회귀해도 개고생이네[완결]', '화산검종', '전설이 되는 법', '풋볼 매니지먼트', '[개정판] 신룡무', '그라니트 : 용들의 땅', '퍼펙트 가이

['코뉴-ONLY ONE LIFE', '태양의 전설 바람의 노래 2부', '디멘션 -게임의 왕', '어둠의 시대', '아티팩트 메이지', '좀비 포식자', '무한 전생(부제-스페이스 니트)', '승소머신 강변호사', '신의 마구', '가면의 시간', '거신사냥꾼', '렙업하는 마왕님', '호감 받고 성공 더!', '포탈 생존기', '검귀의선', '나는 나쁜놈이다', '룰 더 다이아몬드', '대한민국', '행성역사위원회', '역천의 왕']
['서쳐 - 볼수록 능력치 올라', '천마사냥꾼', '더 라이브', '포식마스터', '그땐 득템인 줄 몰랐다', '채널 마스터', '회귀한 프로게이머', '울브즈', '개발자의 게임', '재벌 닷컴', '임해군', '고결한 영혼(Noble Soul)', '만렙튜터 회귀했다', '고교파워클럽', '더 디그 : 유적탐색자', '김 대리가 이렇게 일을 잘했다고?', '악마의 음악 : Other Voices', '언터처블-(개정판)', '프로듀스 좀비군단', '인생역전! 공돌이']
['펜타월드', '데블 & 아이스', '패전처리, 회귀하다', '미친 재능의 마도사', '이질자', '어느 대마도사의 회귀록', '일인전승(一人全勝)', '소공자로 환생한 그랜드마스터', '더 러너 : 마법포식자', '나만 공략법을 알고있다', '자동으로 수련되는 남자', '신석기 마스터', '마법상점의 대마법사', '절세신검존', '폭주상인', '공간지배자 - 이계를 삼키다', '시스템 창 지배자', '혼혈황제의 회귀록', '마수 지배자', '용무쌍']
['트릭아이', 'Hey! 철가방', '황금백수', '샐러리맨과 킬러', 'H.I.D', '죽음의 봄', '21세기 현대 마법전서', '먼치킨의 귀환', '법정에서 봅시다', '꿈꾸는 작사가', '쟁자수', '21세기 마샬아츠', '돌아온 쩔컨 트리오', '랜더 (완결)', '바운더리', '천마님 부활하셨도다', '불사무적', '금검풍운(金劍風雲)', '내나이 90살에 회귀라니!', '용병단장 회귀하다']

['사샤와 레인', '운룡쟁천', '적룡마제', '제라스의 여관', '36.5', '호중지천', '패군', '수라기', '장자지몽', '비룡재천', '테이머 루트', '대형 설서린', '리버스 헌터(Reverse, Rebirth) ', '질풍의 로젠크란츠', '가르딘전기', '무극상도', '이계용사 귀환기', '암제혈로', '내일을 향해 쏴라', '점창파']
['직장은 판타지다.', 'IT제왕 한강수', '듀얼 로드', '블러드 킹', '영광의 포뮬러', '학사 김필도', '엘란', '오메가', '소림삼십칠방', '사상 최강의 영웅(개정판)', '강남무원', '아수라', '역검[易劍]', '우뢰검', '던전 드림', '이계정벌기', '더블커넥션', '천뢰무한', '크로스카운터', '퍼펙트런']
['대법왕', '던전 십팔기', '무림감찰단', '마협 소운강', '무적지존', '마중마제', '데모닉', '천마봉', '부활 쓰는 성기사', '아켄', '고스트아머', '듀얼라이프', '몬스터로드', '궁극의 쉐프', '멋진 인생', '변신술사 카이', '태평천하', '던전 카드사용백서', '리한', '드래곤 남매']
['자베스', '팔부전사', '무적군림보', '신마강림', '오리진 1st', '블러드로드', 'TOP', '철자검', '낭왕', '신존', '내 전투력 10000000000', '광풍무', '연기의 신', '낭인무적', '성모전사', '던전 키퍼(Dungeon Keeper)', '단천붕지', '죽지않는 자(현대 판타지)', '만석꾼', '초월의 플레이어']
['바바리안', '로드 오브 마스터', '지존육성 프로젝트', '풍주', '내 현실은 판타지', '비불전사', '브레이크 마스터', '풍운만리 만학전', '내 안에 몬스터 있다.', '귀향검', '베스트원', '도사 청수', '일로정진', '태극검선', '게이트 러너', '내가 법이다', '질주장가', '천검', '소명전기', '지배하는 자']
['프레어백작', '밀매업자', '마봉추룡기', '혈도제

In [10]:
# 파일 확인하기
df_joara = pd.read_csv('joara.csv')
df_joara

,작품명,연재 플랫폼
0,치트코치 정수찬,조아라
1,조지 부시의 위대한 미국,조아라
2,환생자의 재벌테크,조아라
3,검사 김서진,조아라
4,대표님의 향기로운 덕질생활,조아라
...,...,...
6950,피바라기,조아라
6951,마도군주,조아라
6952,무형객,조아라
6953,영혼의 대장장이[완],조아라
